Please run this script in Google Colab(Or any other IDEs that support iPython).
For 1st use, run all the scripts.
And then you can use function `train(audio_file)` and `gen(message, audio_text)` to generate new audio `fake.wav`.

In [ ]:
!git clone https://github.com/fishaudio/fish-speech.git

# 安装 pytorch
!pip3 install torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1

# (Ubuntu / Debian 用户) 安装 sox + ffmpeg
!apt install libsox-dev ffmpeg

# (Ubuntu / Debian 用户) 安装 pyaudio
!apt install build-essential \
    cmake \
    libasound-dev \
    portaudio19-dev \
    libportaudio2 \
    libportaudiocpp0

# 安裝模型
!huggingface-cli download fishaudio/fish-speech-1.5 --local-dir checkpoints/fish-speech-1.5

# 安装 fish-speech
!pip3 install -e ./fish-speech/
%cd ./fish-speech/

In [ ]:
# 導入語音樣本
import subprocess

%cd ./fish-speech/

def train(filename: str = "../chiang2.mp3"):
    cmd = [
        "python", "tools/vqgan/inference.py",
        "-i", filename,
        "--checkpoint-path", "checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth"
    ]
    subprocess.run(cmd, check=True)

In [18]:
# 用於生成語音
def gen(message: str, 
        prompt: str = "Okay. So you can see compare these two audio signals, uhh you can see the maximum of second signal, uhh you can see the maximum dB is like over minus one.") -> None:
    cmd1 = [
        "python", "tools/llama/generate.py",
        "--text", message,
        "--prompt-text", prompt,
        "--prompt-tokens", "fake.npy",
        "--checkpoint-path", "checkpoints/fish-speech-1.5",
        "--num-samples", "2",
        "--compile"
    ]
    subprocess.run(cmd1, check=True)

    cmd2 = [
        "python", "tools/vqgan/inference.py",
        "-i", "codes_0.npy",
        "--checkpoint-path", "checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth"
    ]
    subprocess.run(cmd2, check=True)


In [19]:
# 使用範例
mes = "Hi, today we'll introduce multimedia."
train()
gen(mes)

And then you'll get a file `fake.wav` in `./fish-speech` folder. This is the file it generate.